# Week 4 Assignment: Create a VGG network

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
class MyClass:
    def __init__(self):        
        self.var1 = 1
        
my_obj = MyClass()

In [4]:
my_obj.__dict__

{'var1': 1}

In [5]:
vars(my_obj)

{'var1': 1}

In [6]:
my_obj.var2 = 2
vars(my_obj)

{'var1': 1, 'var2': 2}

In [7]:
vars(my_obj)['var3'] = 3
vars(my_obj)

{'var1': 1, 'var2': 2, 'var3': 3}

In [8]:
for i in range(4,10):
    # Format a string
    vars(my_obj)[f'var{i}'] = 0
    
vars(my_obj)

{'var1': 1,
 'var2': 2,
 'var3': 3,
 'var4': 0,
 'var5': 0,
 'var6': 0,
 'var7': 0,
 'var8': 0,
 'var9': 0}

In [9]:
# Format a string
i=1
print(f"var{i}")

# Format a string
i=2
print("var{}".format(i))

var1
var2


In [10]:
class MyClass:
    def __init__(self):
        vars(self)['var1'] = 1 

my_obj = MyClass()
vars(my_obj)

{'var1': 1}

In [11]:
# generic VGG block
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions   

        for i in range(repetitions):
            conv = tf.keras.layers.Conv2D(filters, kernel_size, activation="relu", padding="same")    
            vars(self)[f'conv2D_{i}'] = conv
        
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides = (2,2)) 
          
    def call(self, inputs):
        conv2D_0 = vars(self)["conv2D_0"]        
        x = conv2D_0(inputs)

        for i in range(1, self.repetitions):
            conv2D_i = vars(self)[f"conv2D_{i}"]        
            x = conv2D_i(x)

        max_pool = self.max_pool(x)
        return max_pool

In [12]:
# Custom VGG network
class MyVGG(tf.keras.Model):

    def __init__(self, num_classes):
        super(MyVGG, self).__init__()
        self.block_a = Block(64, 3, 2)
        self.block_b = Block(128, 3, 2)
        self.block_c = Block(256, 3, 3)
        self.block_d = Block(512, 3, 3)
        self.block_e = Block(512, 3, 3)

        # Classification head      
        self.flatten = tf.keras.layers.Flatten()        
        self.fc = tf.keras.layers.Dense(256, activation="relu")
        self.classifier = tf.keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

In [ ]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

vgg = MyVGG(num_classes=2)
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224,224))
    return tf.cast(image, tf.float32)/255., features['label']

# transformations
dataset = dataset.map(preprocess).batch(32).prefetch(1)

# vgg.fit(dataset, epochs=15)